<h2>Imports</h2>

In [ ]:
# All the imports

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import types
from botocore.client import Config
import ibm_boto3

import json

import requests
from pandas.io.json import json_normalize

import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium 

<h2>Montreal Raw Data</h2>

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
body = client_55e1b85e93954a0a8adb3f781687106a.get_object(Bucket='capstone-donotdelete-pr-zvspxpvrshowno',Key='quartierreferencehabitation.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
mount = df_data_3.drop(columns=['no_qr', 'nom_mun','no_arr'])

In [ ]:

body = client_55e1b85e93954a0a8adb3f781687106a.get_object(Bucket='capstone-donotdelete-pr-zvspxpvrshowno',Key='Neighbourhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_5 = pd.read_csv(body)


Time to merge!

In [ ]:
# Combine the two data sets
city = mount.merge(df_data_5, how='inner', on='Neighbourhood')

#find one Borough
riv_data = city[city['Borough'] == 'Mercier–Hochelaga-Maisonneuve'].reset_index(drop=True)
riv_data.head()

In [ ]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(riv_data['Borough'].unique()),
        riv_data.shape[0]
    )
)

In [ ]:
address = 'Mercier–Hochelaga-Maisonneuve, QC' 

geolocator = Nominatim(user_agent="qb_explorer")
location = geolocator.geocode(address)
nlatitude = location.latitude
nlongitude = location.longitude
print('The geograpical coordinate of Mercier–Hochelaga-Maisonneuve, Montreal are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Quebec City using latitude and longitude values
map_quebec = folium.Map(location=[nlatitude, nlongitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(riv_data['Latitude'], city['Longitude'], city['Borough'], city['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quebec)  
    
map_quebec

<h2>Using FourSquare API</h2>

<h3>Getting Venues</h3>

In [ ]:
# Foursqaure ID

import json
{
    "tags": [
        "remove-input",
    ]
}
CLIENT_ID = '4F2TWL40HEUEKJXU01ZUMRE4ZNYOGGTRNMPLGRVNJT33J1KJ' # your Foursquare ID
CLIENT_SECRET = 'O5MCRNOS2HOODSSYWX15PYEVB05WZMPY2VGB3GC1K4KQBCFG' # your Foursquare Secret
ACCESS_TOKEN = 'KFUMX2PWRS3FVCUZRGFCYPKLJ1VEKWDLC2NWLHOW24M3S5C4'
version = '20180605' # Foursquare API version
limit = 100 # A default Foursquare API limit value
radius = 2000 

In [ ]:
#function created to find the venues dealing only with food

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        categoryId = '4d4b7105d754a06374d81259' # ID of 'Food'
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            categoryId, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'],
            v['id'],
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                    'Venue Id',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#run function for food venues

city_venues = getNearbyVenues(names=riv_data['Neighbourhood'],
                                   latitudes=riv_data['Latitude'],
                                   longitudes=riv_data['Longitude']
                                  )

In [ ]:
#cleaning up the data duplicates
cityven = city_venues.drop_duplicates(subset=['Venue','Venue Latitude','Venue Longitude'], keep='last')
cityven.head()

In [ ]:
print('There are {} uniques categories.'.format(len(cityven['Venue Category'].unique())))

<h3> Getting Ratings </h3>

In [ ]:
# function for likes of specific venues

def getVenueRatings(venueId):
    
    venues_list=[]

    for venueId in venueId:
        print(venueId)
    
        url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
            venueId, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            version, 
           )    
    
        results = requests.get(url).json()['response']
        venues_list.append((venueId,
            results['likes']['count']))

    near_venues = pd.DataFrame(venues_list, columns =['Venue Id', 'Venue Likes']) 

    return(near_venues)

In [ ]:
# run likes function

venuelist = getVenueRatings(venueId=cityven['Venue Id'])

In [ ]:
#making sure it has collected the right data

venuelist.head()

In [ ]:
#showing the frequency of the likes

venuelist['Venue Likes'].plot(kind='hist')

plt.title('Histogram of Rivière-des-Prairies–Pointe-aux-Trembles Restaurants') # add a title to the histogram
plt.ylabel('Number of Likes') # add y-label
plt.xlabel('Number of Restaurants') # add x-label

plt.show()

In [ ]:
# assigning Categories to the Likes

Poor = venuelist['Venue Likes']<=(np.percentile(venuelist['Venue Likes'], 25))
Acceptable = venuelist[(venuelist['Venue Likes']>(np.percentile(venuelist['Venue Likes'], 25))) & (venuelist['Venue Likes']<=(np.percentile(venuelist['Venue Likes'], 50)))]
Good = venuelist[(venuelist['Venue Likes']>(np.percentile(venuelist['Venue Likes'], 50))) & (venuelist['Venue Likes']<=(np.percentile(venuelist['Venue Likes'], 75)))]
Great = venuelist['Venue Likes']>(np.percentile(venuelist['Venue Likes'], 75))

def conditions(l):
    if l['Venue Likes']<=(np.percentile(venuelist['Venue Likes'], 25)):
        return 'Poor'
    if l['Venue Likes']<=(np.percentile(venuelist['Venue Likes'], 50)):
        return 'Acceptable'
    if l['Venue Likes']<=(np.percentile(venuelist['Venue Likes'], 75)):
        return 'Good'
    if l['Venue Likes']>(np.percentile(venuelist['Venue Likes'], 75)):
        return 'Great'

venuelist['Likes Categories']=venuelist.apply(conditions, axis=1)

In [ ]:
# showing the neighbourhoods, venues and the venue's likes.

venuelike= venuelist.merge(cityven, how='inner', on='Venue Id')
venlike= venuelike.drop(['Neighbourhood Latitude','Neighbourhood Longitude'], axis=1)
venlike.head()

<h2> One SHot this data! </h2> <h3>1s and 0s only</h3>

In [ ]:
# one hot encoding
city_oneshot = pd.get_dummies(venlike[['Likes Categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_oneshot['Venue Id'] = venlike['Venue Id'] 

# move neighborhood column to the first column
fixed_columns = [city_oneshot.columns[-1]] + list(city_oneshot.columns[:-1])
city_oneshot = city_oneshot[fixed_columns]

city_oneshot.head()

In [ ]:
# Order them in frecency of Id

city_grouped = city_oneshot.groupby('Venue Id').mean().reset_index()
city_grouped.head()

<h2>Make a cluster map!</h2>

In [ ]:
kclusters = 4

city_grouped_clustering = city_grouped.drop('Venue Id', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
city_grouped.insert(1, 'Cluster Labels', kmeans.labels_)

# merge tables to add latitude/longitude for each neighborhood
city_merge = city_grouped.merge(venlike, on='Venue Id')

In [ ]:
# create map
latitude = nlatitude
longitude = nlongitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merge['Venue Latitude'], city_merge['Venue Longitude'], city_merge['Neighbourhood'], city_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Cluster "Poor" Rating

city_merge.loc[city_merge['Cluster Labels'] == 0, city_merge.columns[[0] + list(range(6, city_merge.shape[1]))]].head()


In [ ]:
# Cluster "Great" Rating

city_merge.loc[city_merge['Cluster Labels'] == 1, city_merge.columns[[0] + list(range(6, city_merge.shape[1]))]].head()


In [ ]:
# Cluster "Acceptable" Rating

city_merge.loc[city_merge['Cluster Labels'] == 2, city_merge.columns[[0] + list(range(6, city_merge.shape[1]))]].head()


In [ ]:
# Cluster "Good" Rating

city_merge.loc[city_merge['Cluster Labels'] == 3, city_merge.columns[[0] + list(range(6, city_merge.shape[1]))]].head()
